In [1]:
# -*- coding: utf-8 -*-
"""meme_gen.ipynb

Automatically generated by Colaboratory.

"""

import tensorflow as tf
import os

# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))

# strategy = tf.distribute.experimental.TPUStrategy(resolver)

# !git clone https://github.com/TheBrainGamingSingh/BrainGamingMemes.git

# ls

# cd  BrainGamingMemes/

# ls

import json
import datetime
import time
import os
import operator
import numpy as np

# Import Keras
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Dropout, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras import Sequential

# Helper Functions
def map_char_to_int(texts):
    char_counts = {}
    for text in texts:
        for char in text:
            char_counts[char] = char_counts[char] + 1 if char in char_counts else 1
    char_counts_sorted = sorted(char_counts.items(), key=operator.itemgetter(1), reverse=True)
    char_to_int = {}
    for i, row in enumerate(char_counts_sorted):
        char_to_int[row[0]] = i + 1
    return char_to_int

def texts_to_sequences(texts, char_to_int):
    sequences = []
    for text in texts:
        sequences.append([char_to_int[char] for char in text])
    return sequences

# Config

BASE_PATH = os.getcwd()
MODEL_NAME = 'brain_gaming_memes'

try: 
    os.mkdir('models')
except FileExistsError:
    print('models directory already exists...')

MODEL_PATH = BASE_PATH + '/models/' + MODEL_NAME + '_' + datetime.datetime.today().strftime('%Y_%m_%d_%H_%M_%S')

os.mkdir(MODEL_PATH)


SEQUENCE_LENGTH = 128
EMBEDDING_DIM = 16
ROWS_TO_SCAN = 100000
NUM_EPOCHS = 48
BATCH_SIZE = 256

MEME_IDS = [ '114585149',
             '438680',
             '100777631',
             '181913649',
             '161865971',
             '217743513',
             '131087935',
             '119139145',
             '93895088',
             '112126428',
             '97984',
             '1035805',
             '155067746',
             '4087833',
             '91538330',
             '124822590',
             '178591752',
             '124055727',
             '87743020',
             '222403160',
             '102156234',
             '188390779',
             '89370399',
             '129242436']

# Generate input for the model
# Read Training Data
MEME_IDS = [ '100777631']

with open('training_data.json') as file:
    training_data_whole = json.load(file)



training_data = []
for i in training_data_whole:
    if i[0] in MEME_IDS:
        training_data.append(i)
print(f'Number of memes: {len(training_data)}')

texts = []  
labels_index = {}  
labels = []  
label_id_counter = 0

for i, row in enumerate(training_data):
    template_id = str(row[0]).zfill(12)
    text = row[1].lower()
    start_index = len(template_id) + 2 + 1 + 2  # template_id, spaces, box_index, spaces
    box_index = 0
    
    for j in range(0, len(text)):
        char = text[j]
        # note: it is critical that the number of spaces plus len(box_index) is >= the convolution width
        texts.append(template_id + '  ' + str(box_index) + '  ' + text[0:j])
        if char in labels_index:
            label_id = labels_index[char]
        else:
            label_id = label_id_counter
            labels_index[char] = label_id
            label_id_counter += 1
        labels.append(label_id)
        if char == '|':
            box_index += 1

    if i >= ROWS_TO_SCAN:
        break
print(f'Rows to process: {len(texts)}')

del training_data

# Use char to int mapping
char_to_int = map_char_to_int(texts)
sequences = texts_to_sequences(texts, char_to_int)

del texts

# dump the parameters into a json file
with open(MODEL_PATH + '/params.json', 'w') as params:
    json.dump({
        'sequence_length': SEQUENCE_LENGTH,
        'embedding_dim': EMBEDDING_DIM,
        'num_rows_used': len(sequences),
        'num_epochs': NUM_EPOCHS,
        'batch_size': BATCH_SIZE,
        'char_to_int': char_to_int,
        'labels_index': labels_index
    }, params)

data = pad_sequences(sequences, maxlen=SEQUENCE_LENGTH)
del sequences  

labels = np.asarray(labels)

# train and validation split

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels = labels[indices]

# Use a 2% validation split size if rows are more than a million

validation_ratio = 0.2 if data.shape[0] < (10**6) else 0.02
num_validation_samples = int(validation_ratio * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]

x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

del data, labels

print(f'Train Set Size: {x_train.shape[0]}')
print(f'Validation Set Size: {x_val.shape[0]}')

# Model Definition
# with strategy.scope():
model = Sequential()
model.add(Embedding(len(char_to_int) + 1, EMBEDDING_DIM, input_length=SEQUENCE_LENGTH))

model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))

model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))

model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))

model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling1D(2))
model.add(Dropout(0.25))

model.add(Conv1D(1024, 5, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.25))

model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(len(labels_index), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

# make sure we only keep the weights from the epoch with the best accuracy, rather than the last set of weights
checkpoint_handler = ModelCheckpoint(filepath=MODEL_PATH + '/model.h5', verbose=1, save_best_only=True)

model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[checkpoint_handler])



Using TensorFlow backend.
